<a href="https://colab.research.google.com/github/ghoshatanu857/Instrument_Automation/blob/main/Experimental_Applications/LifeTimeMeasurement%20(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install pulsestreamer
# !pip install nidaqmx
import numpy as np
import inspect,dis
import sys
from tqdm import trange
from time import *
# import pulsestreamer
# import pprint
# from pulsestreamer import PulseStreamer,findPulseStreamers,OutputState,TriggerStart,Sequence

In [ ]:
# loading the local Pulse Streamer system
IPaddress = findPulseStreamers(search_serial='')[0][0]
pulser = PulseStreamer(IPaddress)

# loading the local NIDAQmx system
system = nidaqmx.system.System.local()
DAQ_device = system.devices['Dev1']

In [ ]:
# all in nanosecond range
wait1=50; wait2=20
laserOn=1000; Tmax=3000
readOn=150; trigger_timing_delay=2
steps=15; samples=1000; averages=10
laserPort=0; triggerPort=1; timingPort=2

timeRange=np.linspace(0,Tmax,num=steps)

pulse_args = {'wait1': wait1,'wait2': wait2,'laserOn': laserOn,'timeRange': timeRange,'trigger_timing_delay': trigger_timing_delay,
              'readOn': readOn,'averages': averages,'samples': samples,
              'laserPort': laserPort,'triggerPort': triggerPort,'timingPort': timingPort}

In [ ]:
def lifetime_sequence(pulse_args):  # all in nanosecond range

  delay1=pulse_args['wait1']; delay2=pulse_args['wait2']; laser_on =pulse_args['laserOn']; read_on =pulse_args['wait1']
  t_vec =pulse_args['timeRange']; read_on =pulse_args['readOn']; trigegrTimingDelay =pulse_args['trigger_timing_delay']
  averages =pulse_args['averages']; samples =pulse_args['samples']
  laser_port =pulse_args['laserPort']; trigger_port =pulse_args['triggerPort']; timing_port =pulse_args['timingPort']

  # Checking the initial condition of Pulse Streamer
  if pulser.hasSequence()!=0:
    print('Pulse Streamer has a Sequence. Do you want to proceed?\n')
    permission = int(input('Press 0/1 : \n'))
  if permission==0:
    pass

  ## Producing the Patterns
  laser_block = [(delay2,0),(laser_on,1),(delay2+t_vec[-1],0)]
  # laser_pattern = [(delay1,0)]; trigger_pattern = [(delay1,0)]; timing_pattern = [(delay1,0)]

  laser_pattern=[]; trigger_pattern=[]; timing_pattern=[]
  for t in range(t_vec.shape[0]-1): #neglecting the last t
    if t_vec.shape[0] > t_vec[-1]/read_on:
      print(f'Please reduce the step numbers. Maximum allowed steps: {t_vec[-1]/read_on}')
      break
    laserPattern = laser_block*samples
    laser_pattern.append(laserPattern)

    triggerPattern = [((2*delay2+laser_on),0),(read_on,1),(delay2+t_vec[t],0),(read_on,1),(t_vec[-1]+delay2-t_vec[t]-2*(delay2+read_on),0)]
    triggerPattern = triggerPattern*samples
    trigger_pattern.append(triggerPattern)

    timing_read_on = read_on-trigegrTimingDelay
    timingPattern = [((2*delay2+laser_on),0),(timing_read_on,1),(delay2+t_vec[t],0),(timing_read_on,1),(t_vec[-1]+delay2-t_vec[t]-2*(delay2+timing_read_on),0)]
    timingPattern = timingPattern*samples
    timing_pattern.append(timingPattern)

  if t_vec.shape[0] > t_vec[-1]/read_on:
    pass
  # laser_pattern +=[(delay1,0)]; trigger_pattern +=[(delay1,0)]; timing_pattern +=[(delay1,0)]

  ## Producing the Sequences
  reSet = pulser.reset()
  if reSet==0:
    print('Pulse Streamer has been reset before Streaming.\n')
  seq = pulser.createSequence()
  seq.setDigital(laser_port,laser_pattern); seq.setDigital(trigger_port,trigger_pattern); seq.setDigital(timing_port,timing_pattern)
  # print(f'Plot of the Sequence without averaging:\n {seq.plot()}\n')
  # seq*=averages

  return seq

In [ ]:
def lifetime(pulse_args):

     numberofpoints=samples*2
     buffersamplecount=numberofpoints
     DAQ_device.reset_device()

     # Counter
     counter = nidaqmx.Task()
     ciChannel = counter.ci_channels.add_ci_count_edges_chan('/Dev1/ctr1',edge=nidaqmx.constants.Edge.RISING, initial_count=0, count_direction=nidaqmx.constants.CountDirection.COUNT_UP)

     # Trigger
     counter.triggers.pause_trigger.dig_lvl_src='/Dev1/PFI4'
     counter.triggers.pause_trigger.trig_type=nidaqmx.constants.TriggerType.DIGITAL_LEVEL
     counter.triggers.pause_trigger.dig_lvl_when=nidaqmx.constants.Level.LOW

     # Timing
     counter.timing.cfg_samp_clk_timing(rate=1e8,source='/Dev1/PFI5',active_edge=nidaqmx.constants.Edge.FALLING,sample_mode = nidaqmx.constants.AcquisitionType.FINITE, samps_per_chan=(numberofpoints*steps))

     # Pulse streamer Gating
     gate_task = nidaqmx.Task()
     gate_task.do_channels.add_do_chan(lines = 'Dev1/port0/line7')

     # Counter read task
     reader = nidaqmx.stream_readers.CounterReader(counter.in_stream)
     highCount = np.zeros(buffersamplecount, dtype = np.uint32)

     cps = []
     callback=[]

     # Callback function
     def readBuffer(task_handle, every_n_samples_event_type, number_of_samples, callback_data):
         counter.in_stream.read_all_avail_samp = True
         counter.read(number_of_samples_per_channel= nidaqmx.constants.READ_ALL_AVAILABLE)
         cps.extend(highCount)
         callback.extend([1])
         return 0
     counter.register_every_n_samples_acquired_into_buffer_event(buffersamplecount,readBuffer)

     time.sleep(0.01)
     t=0
     run=0
     data=[]
     print("callback number in beginning:",len(callback))

     for i in trange(pulse_args['averages']):

         print(run)
         time.sleep(0.05)
         counter.control(nidaqmx.constants.TaskMode.TASK_RESERVE)
         gate_task.control(nidaqmx.constants.TaskMode.TASK_RESERVE)
         time.sleep(0.05)
         counter.start()

         sequence=lifetime_sequence(pulse_args)
         pulser.setTrigger(start=pulser.TriggerStart.HARDWARE_RISING,rearm=pulser.TriggerRearm.AUTO)

         seq_num=0
         start1=time.time()

         for s in range(steps):

             t1=len(callback)
             seq_num=seq_num+1

             ## Checking the Conditions
             if pulser.hasSequence()!=0:
               print('Pulse Streamer has no Sequence uploaded!')
               pass
             if seq.isEmpty()==0:
               print('Sequence to Pulse Streamer is empty!')
               pass
             if seq.getDuration()%8==0:
               print("Sequence duration is not multiple of 8ns!\n Please check by 'seq.get_pad()' method.")
               pass

             start2=time.time()
             ## Performing the Streaming
             pulser.stream(sequence[s],n_runs=samples,final=([],0,0),start=HARDWARE)
             print(seq_num)

             gate_task.write(True)
             while len(callback)==t1:
                 time.sleep(0.005)
             gate_task.write(False)

             end2=time.time()
             print(f'Time for {s+1}-th average:  {end2-start2}')

         end1=time.time()
         print('Time for single average: ', end1-start1)
         print("callback number average end:",len(callback))
         run=run+1
         counter.control(nidaqmx.constants.TaskMode.TASK_UNRESERVE)
         gate_task.control(nidaqmx.constants.TaskMode.TASK_UNRESERVE)

         if pulser.hasFinished()==1:
           print(f'Streaming has been finished after {i+1}-th average.\n')
         reSet = pulser.reset()
         if reSet==0:
           print('Pulse Streamer has been reset after Streaming.\n')

     data=datainterpretdaq_pulsed_unreserve(cps,numberofpoints*steps)
     counter.close()
     gate_task.close()
     return data

def datainterpretdaq_pulsed_unreserve(combinedcounts,samples_in_one_average):
     combinedcountarray=np.array(combinedcounts)
     averagecount=int(len(combinedcounts)/samples_in_one_average)
     print("crosscheck number of averges=",averagecount)
     combinedcountmatrix=np.reshape(combinedcountarray,(averagecount,samples_in_one_average))
     ca=np.delete(combinedcountmatrix, -1, 1)
     z = np.zeros(averagecount, dtype=int)
     ca2=np.hstack((z[:, np.newaxis], ca))
     correctedcounts=np.subtract(combinedcountmatrix,ca2)
     averagedata=np.mean(correctedcounts,axis=0)
     return averagedata

(array([   0.,  830., 1660., 2490., 3320.]), 1930)

In [ ]:
lifetime(pulse_args)